In [1]:
! git clone https://github.com/google-research/bert

! pip install tf_sentencepiece
! pip install sentencepiece

! wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
! unzip /content/uncased_L-24_H-1024_A-16.zip
! rm /content/uncased_L-24_H-1024_A-16.zip

#https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
#https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
#https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
#https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
#https://storage.googleapis.com/bert_models/2018_10_18/cased_L-24_H-1024_A-16.zip
#https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
#https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

Cloning into 'bert'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 275.96 KiB | 3.68 MiB/s, done.
Resolving deltas: 100% (181/181), done.
     |████████████████████████████████| 2.8MB 3.5MB/s 
     |████████████████████████████████| 1.0MB 3.1MB/s 
--2019-07-09 14:18:24--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 2607:f8b0:4001:c18::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G  63.1MB/s    in 20s     

2019-07-09 14:18:45 (60.7 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]

Archive:  /content/uncased_L-24_H-1024_

In [ ]:
import tensorflow as tf

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import numpy as np
import pandas as pd
from tqdm import tqdm
import collections
import os
import sys
import pprint
import json

import sentencepiece as spm

sys.path.insert(0, '/content/bert')

from run_classifier import FLAGS
FLAGS([None])

import modeling
import optimization
import tokenization

from run_classifier import InputExample, PaddingInputExample, create_model
from run_classifier import file_based_input_fn_builder
from run_classifier import file_based_convert_examples_to_features

tf.logging.set_verbosity(tf.logging.INFO)

def clear_checkpoint_cache():
  '''remove checkpoint files from FLAGS.output_dir (does not remove tf_record files)'''
  files = tf.gfile.ListDirectory(FLAGS.output_dir)
  files = [f for f in files if not f.endswith('tf_record')]
  for file in files:
    tf.gfile.Remove(os.path.join(FLAGS.output_dir, file))
    
def get_lens():
  '''retrieve #_records for train & eval datasets'''
  with open('./dataset_lens.json') as file:
    lens = json.load(file)
    ret = lens['train'], lens['val']
  return ret

In [0]:
from google.colab import auth
auth.authenticate_user()

In [3]:
tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

with tf.Session(tpu_address) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4624636520845270748),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5862292550072502558),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8113030125432651549),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14384874521711867188),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1112809580184861644),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 3079818582861333513),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14918201319546221024),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11325530939983350382),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9260931485257681598),
 _DeviceAttributes(/job:tpu_wor

W0709 18:31:19.954918 139773671593856 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
MODEL_SPEC = 'uncased_L-24_H-1024_A-16'

BUCKET_NAME = ___________

gcs_path = 'gs://' + BUCKET_NAME
pretrained_model_path = gcs_path + '/' + MODEL_SPEC

TRAIN_TFREC_FILE_ = 'train.tf_record'
VAL_TFREC_FILE_ = 'val.tf_record'
TEST_TFREC_FILE_ = 'test.tf_record'

LABEL_LIST = ["0", "1"]

FLAGS.task_name = 'bert_test'
FLAGS.use_tpu = True
FLAGS.num_tpu_cores = 8
FLAGS.tpu_name = tpu_address

FLAGS.do_lower_case = True

FLAGS.max_seq_length = 512

FLAGS.learning_rate = 5e-5
FLAGS.train_batch_size = 32
FLAGS.num_train_epochs = 0.1
FLAGS.warmup_proportion = 0.1

FLAGS.eval_batch_size = 8
FLAGS.predict_batch_size = 8

FLAGS.iterations_per_loop = 1000         # iterations per loop, NOT total train steps
FLAGS.save_checkpoints_steps = 2000      # >= iterations
MAX_SAVED_CHECKPOINTS = 3
EVAL_ALL_CKPT = False

FLAGS.output_dir = gcs_path + '/output'  # tf_record files AND model checkpoints

FLAGS.vocab_file = pretrained_model_path + '/vocab.txt'
FLAGS.bert_config_file = pretrained_model_path + '/bert_config.json' # must be on GCS
FLAGS.init_checkpoint = pretrained_model_path + '/bert_model.ckpt' # must be on GCS

train_tf_rec_file = FLAGS.output_dir + '/' + TRAIN_TFREC_FILE_
val_tf_rec_file = FLAGS.output_dir + '/' + VAL_TFREC_FILE_
test_tf_rec_file = FLAGS.output_dir + '/' + TEST_TFREC_FILE_

In [ ]:
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

if FLAGS.max_seq_length > bert_config.max_position_embeddings:
  raise ValueError(
      "Cannot use sequence length %d because the BERT model "
      "was only trained up to sequence length %d" %
      (FLAGS.max_seq_length, bert_config.max_position_embeddings))

In [0]:
# ONE TIME PER MODEL SPEC: move pretrained model to GS
tf.gfile.MkDir(pretrained_model_path)
model_files = os.listdir('./' + MODEL_SPEC)
for file in model_files:
  os.system('gsutil cp ./' + MODEL_SPEC + '/' + file + ' ' + pretrained_model_path)

In [17]:
# ONE TIME PER FEATURE SET: import data and gen features

def load_data():
  # data load fn
  return train_df, val_df

train_df, val_df = load_data()

print(f'train records: {len(train_df)} val records: {len(val_df)}')

train records: 1325501 val records: 331376


In [ ]:
# ONE TIME PER FEATURE SET: encode inputs & convert to tf_record
#
# In the original bert.run_classifier.py, inputs are converted to features, then 
# saved as tf_record files every time a train run is initiated.  If reusing the
# features already cached to output_dir, skip this step.
#
# Length of each dataset is required for computing train/eval steps, so lengths
# are persisted to local FS as 'dataset_lens.json'.

tokenization.validate_case_matches_checkpoint(FLAGS.do_lower_case,
                                                FLAGS.init_checkpoint)

tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

def create_tf_rec_inputs(df_in, file_out, type=None):
  assert type in ['train', 'val', 'test']
  examples = []
  for (idx, row) in tqdm(df_in.iterrows(), total=len(df_in), mininterval=10):

    if type in ['train', 'val']:
      example = InputExample(idx, row.comment_text, label=row.label)
    elif type == 'test':
      example = InputExample(idx, row.comment_text)

    examples.append(example)

  if type == 'train':
    np.random.shuffle(examples)
  
  if type == 'val':
    # for tpu evaluation, examples must be padded to a multiple of batch size
    while len(examples) % FLAGS.eval_batch_size != 0:
      examples.append(PaddingInputExample())
    assert len(examples) % FLAGS.eval_batch_size == 0
        
  file_based_convert_examples_to_features(examples,
                                          LABEL_LIST,
                                          FLAGS.max_seq_length,
                                          tokenizer,
                                          file_out)
  
  return len(examples)

train_len = create_tf_rec_inputs(train_df, train_tf_rec_file, type='train')
val_len = create_tf_rec_inputs(val_df, val_tf_rec_file, type='val')
#create_tf_rec_inputs(test_df, test_tf_rec_file, type='test')

lens = {'train': train_len,
       'val': val_len}
with open('./dataset_lens.json', 'w') as file:
  json.dump(lens, file)

In [ ]:
! gsutil ls $pretrained_model_path

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_address)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    keep_checkpoint_max=MAX_SAVED_CHECKPOINTS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [0]:
# from bert.run_classifier.py -- added AUC to metric_fn

def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        auc = tf.metrics.auc(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        return {
            "eval_accuracy": accuracy,
            "eval_auc": auc,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn,
                      [per_example_loss, label_ids, logits, is_real_example])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [9]:
train_len, val_len = get_lens()
print('train len', train_len, 'val len', val_len)

num_train_steps = int(train_len / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
print('num_train_steps', num_train_steps)

model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(LABEL_LIST),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)

train len 1325501 val len 331376
num_train_steps 4142


In [ ]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    predict_batch_size=FLAGS.predict_batch_size,
    eval_batch_size=FLAGS.eval_batch_size)

In [ ]:
train_input_fn = file_based_input_fn_builder(
  input_file=train_tf_rec_file,
  seq_length=FLAGS.max_seq_length,
  is_training=True,
  drop_remainder=True)

In [12]:
clear_checkpoint_cache()
tf.gfile.ListDirectory(FLAGS.output_dir)

['train.tf_record', 'val.tf_record']

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

In [0]:
eval_input_fn = file_based_input_fn_builder(
    input_file=val_tf_rec_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=True if FLAGS.use_tpu else False)

In [ ]:
steps_and_files = []
filenames = tf.gfile.ListDirectory(FLAGS.output_dir)

for filename in filenames:
  if filename.endswith(".index"):
    ckpt_name = filename[:-6]
    cur_filename = os.path.join(FLAGS.output_dir, ckpt_name)
    global_step = int(cur_filename.split("-")[-1])
    tf.logging.info("Add {} to eval list.".format(cur_filename))
    steps_and_files.append([global_step, cur_filename])
steps_and_files = sorted(steps_and_files, key=lambda x: x[0])

if not EVAL_ALL_CKPT:
  steps_and_files = steps_and_files[-1:]

In [ ]:
eval_results = []
eval_steps = val_len // FLAGS.eval_batch_size
print('eval steps', eval_steps)

for global_step, filename in sorted(steps_and_files, key=lambda x: x[0]):
  ret = estimator.evaluate(
      input_fn=eval_input_fn,
      steps=eval_steps,
      checkpoint_path=filename)
  
  ret["step"] = global_step
  ret["path"] = filename

  eval_results.append(ret)

  tf.logging.info("=" * 80)
  log_str = "Eval result | "
  for key, val in sorted(ret.items(), key=lambda x: x[0]):
    log_str += "{} {} | ".format(key, val)
  tf.logging.info(log_str)

In [26]:
eval_results

[{'eval_accuracy': 0.94855994,
  'eval_auc': 0.9138441,
  'eval_loss': 0.23178084,
  'global_step': 4142,
  'loss': 0.2222023,
  'path': 'gs://bert422/output/model.ckpt-4142',
  'step': 4142}]